# Running hofx application in JEDI

## prerequisite 
- `jedi-bundle` is already built in your `JEDI_BUILD`.
- Background files
- observation files
- Input geometry files


---

## Introduciton:

An H(x) (or hofx) application computes the model values at observation time/location. Depending on the observation type, this task can be as simple as extracting model values at the observation location. If the observed variable does not exist in the model outputs, it can be calculated using "observation operators" in the JEDI Unified Forward Operator (UFO) code. In the example here, the observed variable is Aerosol Optical Depth (AOD), which is not usually available in the models. The AOD observation operator will calculate AOD from the model state (using CRTM) to provide it to JEDI. 

There are two main approaches for computing H(x) in JEDI: 
- HofX3D: computes H(x) using model states (x) already available to the application. The application assumes all the observations within the assimilation window are valid in the middle of the window. Thus, the model state must be valid in the middle of the assimilation window. The JEDI executable for this application is `fv3jedi_hofx_nomodel.x`
  
- HofX4D: computes H(x) using model states computed by running the forecast model (can be a full non-linear model, a simplified linear model, pseudo, etc.). It requires an initial model state valid at the beginning of the assimilation window. The application will move the initial state forward using the model specified by the user. Users can set `tstep` to specify the length of the "subwindow" in the assimilation window. Observations within each subwindow are assumed to be valid at the beginning of that subwindow, which is the same time as the model state. (is this correct???). The JEDI executable for this application is `fv3jedi_hofx.x`



## YAML structure (HofX3D):

```yaml
# Beginning and length of assimilation window
time window:
  begin: 2021-08-04T21:00:00Z
  length: PT6H

# Geometry of the model
geometry:
  fms initialization:
    namelist filename: geometry_input/fmsmpp.nml
  akbk: geometry_input/akbk72.nc4
  npx: 13
  npy: 13
  npz: 72
  field metadata override: geometry_input/geos.yaml

# Model state valid in the middle of the assimilation window
state:
  datetime: 2021-08-05T00:00:00Z
  filetype: cube sphere history
  datapath: geos_c12
  filenames: [aero_x48rt.abkg.eta.%yyyy%mm%dd_%hh%MMz.nc4, met_x48rt.bkg_clcv.%yyyy%mm%dd_%hh%MMz.nc4]
  state variables: [ud,vd,ua,va,t,delp,q,qi,ql,phis,SO4,
                    BCPHOBIC,BCPHILIC,BRPHOBIC,BRPHILIC,
                    DU,DU002,DU003,DU004,DU005,
                    SS,SS002,SS003,SS004]
# Observations measured throughout the assimilation window
# But are assumed to be valid at the middle of the assimilation window.
observations:
  observers:
  - obs space:
      name: Aod
      obsdatain:
        engine:
          type: H5File
          obsfile: obs/aod_viirs_obs_202108042100.nc4
      obsdataout:
        engine:
          type: H5File
          obsfile: output/fb.3d.aod_viirs_obs_4dvar_geos_aero_202108042100.nc4
      simulated variables: [aerosolOpticalDepth]
      channels: 4
    obs operator:
      name: AodCRTM
      Absorbers: [H2O,O3]
      obs options:
        Sensor_ID: v.viirs-m_j1
        EndianType: little_endian
        CoefficientPath: crtm/
        AerosolOption: aerosols_gocart_default
        model units coeff: 1.0e9
    obs error:
      covariance model: diagonal
    get values:
      time interpolation: linear
```

## YAML structure (HofX4D):

You can use ctest `fv3jedi_test_tier1_hofx_fv3-geos_cf` in [fv3-jedi](https://github.com/JCSDA-internal/fv3-jedi/blob/1.8.0/test/CMakeLists.txt#L1107) as an [example](https://github.com/JCSDA-internal/fv3-jedi/blob/1.8.0/test/testinput/hofx_fv3-geos_cf.yaml). Here, I explain how to run an stand-alone hofx run. 

## YAML structure:

An example of an input YAML file is below. The user needs to specify:

- Time window: 



```yaml
time window:
  begin: 2020-09-03T15:00:00Z
  length: PT6H
forecast length: PT6H

geometry:
  fms initialization:
    namelist filename: geometry_input/fmsmpp.nml
  akbk: geometry_input/akbk72.nc4
  npx: 13
  npy: 13
  npz: 72
  field metadata override: geometry_input/geos_cf.yaml
  
model:
  name: PSEUDO
  filetype: cube sphere history
  datapath: geos_c12
  filename: geos_cf.bkg.%yyyy%mm%dd_%hh%MM%ssz.nc4
  tstep: PT3H

initial condition:
  datetime: 2020-09-03T15:00:00Z
  filetype: cube sphere history
  datapath: geos_c12
  filename: geos_cf.bkg.%yyyy%mm%dd_%hh%MM%ssz.nc4
  state variables: [DELP,volume_mixing_ratio_of_no2,volume_mixing_ratio_of_no,volume_mixing_ratio_of_o3,PS,SPHU]
  
observations:
  observers:
  - obs space:
      name: NO2
      obsdatain:
        engine:
          type: H5File
          obsfile: obs/tropomi_no2_tropo_2020090318_m.nc4
      obsdataout:
        engine:
          type: H5File
          obsfile: output/fb.tropomi_no2_geos_2020090318_m.nc
      simulated variables: [nitrogendioxideColumn]
    obs operator:
      name: ColumnRetrieval
      nlayers_retrieval: 34
      tracer variables: [volume_mixing_ratio_of_no2]
      isApriori: false
      isAveragingKernel: true
      stretchVertices: topbottom #options: top, bottom, topbottom, none
    obs error:
      covariance model: diagonal
```

---
## Run the application

First, set the path to your `JEDI_BUILD` and `MPIEXEC`. For example for me on Discover, it looks like this:

```bash
export MPIEXEC=/discover/swdev/jcsda/spack-stack/openmpi-4.1.3/gcc-10.1.0/bin/mpiexec
export JEDI_BUILD=/discover/nobackup/mabdiosk/jedi-bundle/build-gnu-spack-1.5.1
```

All the files that you need are under `hofx` directory. So `cd hofx` and then you can run the application this command: 

```bash
$MPIEXEC "-n" "6" $JEDI_BUILD/bin/fv3jedi_hofx.x hofx_fv3-geos_cf.yaml
```

In [ ]:
September 3, 2020 6:39:39 PM
to
September 3, 2020 6:42:26 PM

Obs within 18Z

time window: 2020-09-03T15:00:00Z